In [15]:
import glob
from collections import defaultdict
import pandas as pd
import os.path as op
import math
import os 
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import sqlite3
import sqlalchemy

def safe_makedir(dname):
    """
    Make a directory if it doesn't exist, handling concurrent race conditions.
    """
    if not dname:
        return dname
    num_tries = 0
    max_tries = 5
    while not os.path.exists(dname):
        try:
            os.makedirs(dname)
        except OSError:
            if num_tries > max_tries:
                raise
            num_tries += 1
            time.sleep(2)
    return dname

# import my custom functions
sys.path.insert(0, '/home/jmunson-mcgee/')
from JMM_functions import *

pd.set_option("display.max_rows", 6)

outdir='/mnt/scgc/simon/microg2p/Melody/BLM1/Analyses/RNA_GORG_recruit/Summary_files/'
safe_make_dir(outdir)
os.chdir(outdir)

    
matplotlib.__version__

read_files = glob.glob('/mnt/scgc/simon/microg2p/Melody/BLM1/Analyses/RNA_GORG_recruit/Summary_files/*annotated_RNA_GORG_results.csv')


Folder already exists


# First steps
1. Use the 10none and 500none recruitments to calculate how many cells/ml there are for genera/families with at least 5 cells that are 20% complete or more
1. Calculate the number of reads per gene for each sample (this is probably more of an RNA thing but oh well.

In [16]:
SAGdata='/mnt/scgc/simon/microg2p/analyses/JdF_analysis/summary_files/JdF_SAGs_GTDB_2.0_and_95ANI_clustering_master_predresp_rate.csv'

In [17]:
# cols=['name', 'GTDB_classification', 'checkM_estimated_completeness', 'hypothetical_cds_fraction', 'genus', 'family', 'depth']

# JdF_SAGs=pd.read_csv(SAGdata)#, usecols=cols)
# JdF_SAGs.columns

In [18]:
cols=['name', 'GTDB_classification', 'final_assembly_length', 'checkM_estimated_completeness', 'hypothetical_cds_fraction', 'genus', 'family', 'depth']

JdF_SAGs=pd.read_csv(SAGdata)#, usecols=cols)
# JdF_SAGs=JdF_SAGs[JdF_SAGs['checkM_estimated_completeness']>=20]

JdF_genus_counts=JdF_SAGs.groupby('genus', as_index=False)['hypothetical_cds_fraction'].count()
JdF_genus_counts.rename(columns = {'hypothetical_cds_fraction':'number_of_cells'}, inplace=True)

JdF_genus_stats=JdF_SAGs.groupby('genus', as_index=False)[['hypothetical_cds_fraction', 'checkM_estimated_completeness', 'final_assembly_length']].mean()
JdF_genus_stats=JdF_genus_stats.merge(JdF_genus_counts, on='genus', how='left')

JdF_genus_stats['genus']=JdF_genus_stats['genus'].str.strip()
print(len(JdF_genus_counts))



JdF_family_counts=JdF_SAGs.groupby('family', as_index=False)['hypothetical_cds_fraction'].count()
JdF_family_counts.rename(columns = {'hypothetical_cds_fraction':'number_of_cells'}, inplace=True)

JdF_family_stats=JdF_SAGs.groupby('family', as_index=False)[['hypothetical_cds_fraction', 'checkM_estimated_completeness', 'final_assembly_length']].mean()
JdF_family_stats=JdF_family_stats.merge(JdF_family_counts, on='family', how='left')

JdF_family_stats['family']=JdF_family_counts['family'].str.strip()
# JdF_family_counts=JdF_family_counts[JdF_family_counts['number_of_cells']>=5]


print(len(JdF_family_stats))
JdF_family_stats

168
99


,family,hypothetical_cds_fraction,checkM_estimated_completeness,final_assembly_length,number_of_cells
0,2-12-FULL-41-12,0.397177,43.180000,5.016050e+05,1
1,AAA536-G10,0.244515,27.107222,6.986373e+05,36
2,AG-339-G14,0.233311,36.849000,6.941456e+05,10
...,...,...,...,...,...
96,UBA890,0.364000,10.340000,5.349430e+05,1
97,UBA9320,0.342016,44.830000,1.026203e+06,1
98,Woeseiaceae,0.251808,16.718000,5.169096e+05,5


This leaves me with 50 genera and 36 families (including Unclassified) that have at least 5 cells with a checkm completeness of at least 20%. 


I still need to find my file that has the total number of reads per sample.

In [19]:
# This step takes a while so I ran it once and then hardocded the output into the variables for the number of reads and comment out the count commands
# ! zcat /mnt/scgc/simon/microg2p/JdF_analysis/analysis/JdF_GORG_RNA_recruit/annotations/All_10none_reads_noMTST_annotated.txt.gz | wc -l
# ! zcat /mnt/scgc/simon/microg2p/JdF_analysis/analysis/JdF_GORG_RNA_recruit/annotations/All_500none_reads_noMTST_annotated.txt.gz | wc -l


All10_reads=106908161
All500_reads=84370726


In [20]:
for file in read_files:
    base=os.path.basename(file)
    base=base[:-11]
    Genus_name=base+'Genus_pivot.csv'
    Family_name=base+'Family_pivot.csv'
    Order_name=base+'Order_pivot.csv'
    print(base)
    
    # I need to modify this so that I have one file that is a pivot of reads classified to the genus and one of reads classified to the fmaily level
    
    DF=pd.read_csv(file, index_col=0)
    
    if len (DF)==0:
        continue
    
    else:
        DF['taxonomic_lineage']=DF['taxonomic_lineage'].str[:-1]

        DF[['domain', 'phyla', 'class', 'order', 'family', 'genus', 'species']]=DF.taxonomic_lineage.str.split(';', n=7, expand=True)
        DF_Genus=DF.pivot_table(index='genus', columns='gene_cluster_rep', values='number_of_reads', aggfunc=np.sum, fill_value=0)
        DF_Genus.to_csv(os.path.join(outdir, Genus_name))
    
        DF_Family=pd.pivot_table(DF, index='gene_cluster_rep', columns='family', values='number_of_reads', aggfunc=np.sum, fill_value=0)
        DF_Family.to_csv(os.path.join(outdir, Family_name))
        
        DF_Order=pd.pivot_table(DF, index='gene_cluster_rep', columns='order', values='number_of_reads', aggfunc=np.sum, fill_value=0)
        DF_Order.to_csv(os.path.join(outdir, Order_name))
        
DF_Order



cDNA4-A01_joined_annotated_RNA_GORG_
cDNA2-A01_joined_annotated_RNA_GORG_
cDNA3-A01_joined_annotated_RNA_GORG_
cDNA7-A01_joined_annotated_RNA_GORG_
cDNA1-A01_joined_annotated_RNA_GORG_


order,Aminicenantales,B26-1,Bryobacterales,Carboxydocellales,Hadarchaeales,Methanomassiliicoccales,NA,RBG-16-55-9,Thermodesulfovibrionales
gene_cluster_rep,,,,,,,,,
AM-294-J06_N23;5746;6825,0,0,0,1,0,0,0,0,0
AM-294-K13_N4;7685;8899,0,0,0,0,0,0,46,0,18569
AM-294-L07_N4;47996;48265,0,15,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
AM-302-A23_N6;3581;4708,0,0,0,0,0,0,0,46,0
AM-304-B23_N71;959;1417,0,0,0,0,0,0,0,0,1
AM-307-C17_N9;25035;25322,0,0,0,0,0,0,0,0,24331


In [21]:
test=pd.read_csv('/mnt/scgc/simon/microg2p/Melody/BLM1/Analyses/RNA_GORG_recruit/Summary_files/cDNA1-A01_joined_annotated_RNA_GORG_results.csv')
test['taxonomic_lineage']=test['taxonomic_lineage'].str[:-1]
test[['domain', 'phyla', 'class', 'order', 'family', 'genus', 'species']]=test.taxonomic_lineage.str.split(';', expand=True)

test


,Unnamed: 0,taxonomic_lineage,gene_cluster_rep,number_of_reads,prokka_EC_number,prokka_product,swissprot_gene,swissprot_EC_number,swissprot_eggNOG,swissprot_KO,swissprot_Pfam,swissprot_CAZy,swissprot_TIGRFAMs,domain,phyla,class,order,family,genus,species
0,0,Archaea; Hadarchaeota; Hadarchaeia; Hadarchaea...,AM-294-L10_N20;891;1535,7457.0,NaN,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Archaea,Hadarchaeota,Hadarchaeia,Hadarchaeales,WYZ-LMO6,NA,NA
1,1,Archaea; Thermoplasmatota; Thermoplasmata; Met...,AM-294-P13_N14;17572;18009,8336.0,NaN,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Archaea,Thermoplasmatota,Thermoplasmata,Methanomassiliicoccales,JACIVX01,NA,JACIVX01 sp011334735
2,2,Bacteria; Acidobacteriota; Aminicenantia; Amin...,AM-294-P18_N23;17116;17334,32802.0,NaN,Putative nitrogen fixation protein YutI,NaN,NaN,"ENOG41083MR,CCOG0694",NaN,PF01106,NaN,NaN,Bacteria,Acidobacteriota,Aminicenantia,Aminicenantales,DTGS01,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,13,Bacteria; Acidobacteriota; Aminicenantia; Amin...,AM-294-P18_N23;17116;17334,1.0,NaN,Putative nitrogen fixation protein YutI,NaN,NaN,"ENOG41083MR,CCOG0694",NaN,PF01106,NaN,NaN,Bacteria,Acidobacteriota,Aminicenantia,Aminicenantales,Saccharicenantaceae,Saccharicenans,NA
14,14,Bacteria; Firmicutes_B; GCA-003054495; Carboxy...,AM-294-J06_N23;5746;6825,1.0,6.3.5.5,Carbamoyl-phosphate synthase pyrimidine-specif...,NaN,NaN,NaN,NaN,"PF00988,CPF00117",NaN,TIGR01368,Bacteria,Firmicutes_B,GCA-003054495,Carboxydocellales,Carboxydocellaceae,Carboxydocella,Carboxydocella thermautotrophica
15,15,Bacteria; Nitrospirota; Thermodesulfovibrionia...,AM-296-B22_N14;2166;2447,1.0,NaN,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bacteria,Nitrospirota,Thermodesulfovibrionia,Thermodesulfovibrionales,NA,NA,NA
